# FHIR Inferno Test Generator

This notebook generates executable Inferno test files from FHIR Implementation Guide test specifications. The output consists of Ruby tests that can be used to validate FHIR server implementations.

#### What it does

- Processes test specifications from a consolidated test plan
- Analyzes requirements and transforms them into executable test code
- Generates comprehensive Ruby tests that follow the Inferno testing framework structure
- Creates a complete, organized test kit with proper file structure

#### How to use

1. **Setup**: Individual cert setup may need to be modified in `setup_clients()` function of the llm_utils.py file before running this notebook. API keys should be in .env file. Make sure you have an API key for at least one of:
   - Anthropic Claude (`ANTHROPIC_API_KEY`)
   - Google Gemini (`GEMINI_API_KEY`) 
   - OpenAI GPT-4 (`OPENAI_API_KEY`)

2. **Input**: A markdown file with test specifications in the following format:
   ```markdown
   ### REQ-ID: Requirement Title

   **Description**: "Detailed description of the requirement"

   **Actor**: System component responsible

   **Conformance**: SHALL/SHOULD/MAY

   # Test Specification for REQ-ID
   ... detailed test specification ...

### Notebook Setup

In [262]:
# FHIR Inferno Test Generator
import re
import os
import logging
import time
import json
import httpx
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
from collections import defaultdict

import pandas as pd
from dotenv import load_dotenv

# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Constants
CURRENT_DIR = Path.cwd()  # Current working directory
OUTPUT_DIR = os.path.join(CURRENT_DIR, 'test_output')
PROJECT_ROOT = Path.cwd().parent  # Go up one level to project root


In [263]:
import importlib.util
module_path = os.path.join(PROJECT_ROOT, 'llm_utils.py')

spec = importlib.util.spec_from_file_location("llm_utils", module_path)
llm_utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(llm_utils)

In [ ]:
# Import prompt utilities
prompt_utils_path = os.path.join(PROJECT_ROOT, 'prompt_utils.py')
spec = importlib.util.spec_from_file_location("prompt_utils", prompt_utils_path)
prompt_utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(prompt_utils)

# Setup the prompt environment
prompt_env = prompt_utils.setup_prompt_environment(PROJECT_ROOT)
PROMPT_DIR = prompt_env["prompt_dir"]
TEST_GEN_PATH = prompt_env["test_gen_path"]

logging.info(f"Using prompts directory: {PROMPT_DIR}")
logging.info(f"Inferno test generation prompt: {TEST_GEN_PATH}")

### API Prompts

In [ ]:
# System prompts for test generation
INFERNO_TEST_SYSTEM_PROMPT = """You are a specialized FHIR testing engineer with expertise in healthcare interoperability.
Your task is to convert test specifications from a test plan into executable Ruby tests using the Inferno testing framework.
You will generate valid, working Ruby code that follows Inferno test patterns and best practices."""

In [ ]:
def get_inferno_test_generation_prompt(test_specification: str, requirement_id: str, module_name: str, inferno_guidance: str) -> str:
    """
    Load the Inferno test generation prompt from file and format it with test details
    
    Args:
        test_specification: The test specification content
        requirement_id: The ID of the requirement
        module_name: The name of the module
        inferno_guidance: The Inferno guidance
        
    Returns:
        str: The formatted prompt for the LLM
    """
    return prompt_utils.load_prompt(
        TEST_GEN_PATH,
        test_specification=test_specification,
        requirement_id=requirement_id,
        module_name=module_name,
        inferno_guidance=inferno_guidance
    )

### Main Processing Functions

In [265]:
def parse_test_plan(file_path: str) -> Dict[str, Any]:
    """
    Parse a consolidated test plan into sections and requirements
    focusing on the main requirements
    """
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Initialize empty sections dictionary
    sections = {}
    
    # First try to extract sections from the TOC
    toc_pattern = r'## Table of Contents\n(.*?)(?:\n## |\n# |\Z)'
    toc_match = re.search(toc_pattern, content, re.DOTALL)
    
    if toc_match:
        toc_content = toc_match.group(1)
        # Extract section names from the TOC
        section_pattern = r'- \[(.*?)\]'
        section_matches = re.findall(section_pattern, toc_content)
        
        # Only use top-level sections (not the REQ items)
        top_level_sections = [s for s in section_matches if not s.startswith('REQ-')]
        
        # Create section entries for each section found
        for section_name in top_level_sections:
            clean_section_name = section_name.strip()
            section_id = clean_section_name.lower().replace(' ', '-')
            sections[clean_section_name] = {
                'id': section_id,
                'name': clean_section_name,
                'content': "",
                'requirements': []
            }
    
    # If no sections found in TOC, try to find sections from the document headings
    if not sections:
        # Look for section headers (##) that are not part of the TOC
        section_pattern = r'## ([^\n#]+)(?=\n(?!## Table of Contents))'
        section_matches = re.findall(section_pattern, content)
        
        for section_name in section_matches:
            clean_section_name = section_name.strip()
            # Skip if this is a "Test Specification" section
            if "Test Specification" in clean_section_name:
                continue
                
            section_id = clean_section_name.lower().replace(' ', '-')
            sections[clean_section_name] = {
                'id': section_id,
                'name': clean_section_name,
                'content': "",
                'requirements': []
            }
    
    # Find all requirement headers in the document
    req_pattern = r'### (REQ-\d+): (.*?)\n\n\*\*Description\*\*: "(.*?)"\n\n\*\*Actor\*\*: (.*?)\n\n\*\*Conformance\*\*: (.*?)(?:\n\n|$)'
    req_matches = re.findall(req_pattern, content, re.DOTALL)
    
    print(f"Found {len(req_matches)} potential requirements")
    
    # Process each requirement
    for req_id, req_title, req_desc, req_actor, req_conf in req_matches:
        print(f"Processing requirement: {req_id}")
        
        # Find the full test specification for this requirement
        test_spec_pattern = f"# Test Specification for {req_id}(.*?)(?:---|\\n## |$)"
        test_spec_match = re.search(test_spec_pattern, content, re.DOTALL)
        test_spec = test_spec_match.group(1).strip() if test_spec_match else ""
        
        # Try to determine which section this requirement belongs to
        assigned_to_section = False
        
        # First try to find the requirement in one of the identified sections
        for section_name, section in sections.items():
            section_start = content.find(f"## {section_name}")
            if section_start == -1:
                continue
                
            # Find the next section to determine where this section ends
            next_section_start = len(content)
            for other_section in sections.keys():
                if other_section != section_name:
                    other_start = content.find(f"## {other_section}", section_start + len(section_name))
                    if other_start > section_start and other_start < next_section_start:
                        next_section_start = other_start
            
            # Extract the content of this section
            section_content = content[section_start:next_section_start]
            
            if f"### {req_id}" in section_content:
                # This requirement belongs to this section
                requirement = {
                    'id': req_id,
                    'title': req_title.strip(),
                    'description': req_desc.strip(),
                    'actor': req_actor.strip(),
                    'conformance': req_conf.strip(),
                    'full_content': f"### {req_id}: {req_title}\n\n**Description**: \"{req_desc}\"\n\n**Actor**: {req_actor}\n\n**Conformance**: {req_conf}",
                    'full_spec': test_spec,
                    'section': section_name,
                    'testability': 'Automatic'  # Default value
                }
                
                sections[section_name]['requirements'].append(requirement)
                print(f"Added requirement {req_id} to section {section_name}")
                assigned_to_section = True
                break
        
        # If we couldn't find the requirement in any of our sections,
        # create a section based on the actor if it doesn't exist already
        if not assigned_to_section:
            actor = req_actor.strip()
            if not actor:
                actor = "Unknown Actor"
                
            if actor not in sections:
                section_id = actor.lower().replace(' ', '-')
                sections[actor] = {
                    'id': section_id,
                    'name': actor,
                    'content': "",
                    'requirements': []
                }
            
            requirement = {
                'id': req_id,
                'title': req_title.strip(),
                'description': req_desc.strip(),
                'actor': actor,
                'conformance': req_conf.strip(),
                'full_content': f"### {req_id}: {req_title}\n\n**Description**: \"{req_desc}\"\n\n**Actor**: {req_actor}\n\n**Conformance**: {req_conf}",
                'full_spec': test_spec,
                'section': actor,
                'testability': 'Automatic'  # Default value
            }
            
            sections[actor]['requirements'].append(requirement)
            print(f"Added requirement {req_id} to actor-based section {actor}")
    
    # Remove empty sections
    sections = {k: v for k, v in sections.items() if v['requirements']}
    
    return sections

In [266]:
def get_inferno_guidance() -> str:
    """
    Load the Inferno guidance document
    
    Returns:
        String containing Inferno test development guidance
    """
    guidance_path = os.path.join(PROJECT_ROOT, 'inferno-guidance.md')
    
    # Use a default guidance if file not found
    if not os.path.exists(guidance_path):
        return """# Inferno Test Development Guidance
        
        Inferno is a Ruby-based testing framework for FHIR implementations. Tests should follow the structure:
        
        ```ruby
        module YourTestKit
          class YourTestGroup < Inferno::TestGroup
            id :unique_id
            title 'Test Group Title'
            description 'Detailed description'
            
            test do
              id :test_unique_id
              title 'Test Title'
              description 'Test description'
              
              run do
                # Test implementation
                assert condition, 'Failure message'
              end
            end
          end
        end
        ```
        """
    
    with open(guidance_path, 'r') as f:
        return f.read()

In [267]:
def determine_test_groups(sections: Dict[str, Dict[str, Any]], expected_actors: List[str]) -> Dict[str, Dict[str, List[Dict[str, Any]]]]:
    """
    Group test specifications based on the sections and actors from the test plan
    
    Args:
        sections: Dictionary of sections from the parsed test plan
        expected_actors: List of expected actors from user input
        
    Returns:
        Nested dictionary of grouped test specifications
    """
    # First, group by actor
    actor_groups = defaultdict(list)
    
    # Create a mapping from normalized actor names to expected actor names
    actor_mapping = {actor.lower().replace(' ', '_'): actor for actor in expected_actors}
    
    # Collect all requirements across all sections
    for section_name, section in sections.items():
        for req in section['requirements']:
            # Use the exact actor name from the requirement
            actor = req['actor'].strip()
            if not actor:
                actor = "Unknown Actor"
            else:
                # Try to match with expected actors (case-insensitive)
                actor_lower = actor.lower().replace(' ', '_')
                if actor_lower in actor_mapping:
                    actor = actor_mapping[actor_lower]
            
            # Store the requirement with its section information
            req_with_section = req.copy()
            req_with_section['section_name'] = section_name
            req_with_section['section_id'] = section['id']
            
            actor_groups[actor].append(req_with_section)
    
    # Only include actors that have requirements
    result_groups = {}
    for actor, reqs in actor_groups.items():
        if reqs:  # Only include non-empty actor groups
            if actor not in result_groups:
                result_groups[actor] = {}
            
            # Group by original section
            section_groups = defaultdict(list)
            for req in reqs:
                section_groups[req['section_name']].append(req)
            
            # Add the section groups to the result
            result_groups[actor] = dict(section_groups)
    
    return result_groups

In [ ]:
def generate_tests_for_section(
    client, 
    api_type: str,
    section: Dict[str, Any],
    inferno_guidance: str,
    module_name: str,
    rate_limit_func,
    max_token_limit: int = 16000
):
    """
    Generate tests for an entire section or individual requirements based on token limits
    
    Args:
        client: The API client
        api_type: API type (claude, gemini, gpt)
        section: Section dictionary containing requirements
        inferno_guidance: Inferno test development guidance
        module_name: Name of the module for the test
        rate_limit_func: Function to check rate limits
        max_token_limit: Maximum tokens for the model
        
    Returns:
        Dictionary mapping requirement IDs to generated tests
    """
    logger.info(f"Generating tests for section: {section['name']}")
    
    # First, try to generate tests for the entire section
    if len(section['requirements']) > 1:
        # Construct a prompt for the entire section
        section_prompt = f"""
        Generate Inferno tests for the following section of requirements from a FHIR implementation guide.
        
        Section: {section['name']}
        
        Requirements:
        {section['content']}
        
        For each requirement, create a separate Inferno test class following the naming convention:
        - Class name: {module_name}[Actor][Resource][REQ-ID]Test
        - File name: req_[id]_test.rb
        
        Module Name: {module_name}
        
        Follow this Inferno development guidance:
        {inferno_guidance}
        
        Return the Ruby code for each test implementation, separated by clear markers indicating the requirement ID.
        """
        
        # Estimate token count (rough approximation)
        estimated_tokens = len(section_prompt) / 4  # Approximate 4 chars per token
        
        if estimated_tokens < max_token_limit:
            try:
                logger.info(f"Attempting to generate tests for entire section: {section['name']}")
                response = llm_utils.make_llm_request(
                    client, 
                    api_type, 
                    section_prompt, 
                    INFERNO_TEST_SYSTEM_PROMPT, 
                    rate_limit_func
                )
                
                # Parse the response to extract individual tests
                tests = {}
                current_req = None
                current_test = []
                
                for line in response.split('\n'):
                    # Look for markers indicating requirement boundaries
                    req_marker = re.search(r'(REQ-\d+)', line)
                    if req_marker and ("Test for" in line or "Implementation for" in line or "# Requirement" in line):
                        if current_req and current_test:
                            tests[current_req] = '\n'.join(current_test)
                            current_test = []
                        
                        current_req = req_marker.group(1)
                    
                    if current_req:
                        current_test.append(line)
                
                # Don't forget the last test
                if current_req and current_test:
                    tests[current_req] = '\n'.join(current_test)
                
                # If we successfully parsed tests for all requirements, return them
                if len(tests) == len(section['requirements']):
                    logger.info(f"Successfully generated tests for all requirements in section: {section['name']}")
                    return tests
                
                logger.warning(f"Generated tests for only {len(tests)} of {len(section['requirements'])} requirements in section: {section['name']}")
                # Continue to individual requirement processing
            
            except Exception as e:
                logger.warning(f"Failed to generate tests for entire section: {str(e)}")
                # Continue to individual requirement processing
    
    # If section-level generation failed or wasn't attempted, generate tests for individual requirements
    tests = {}
    for requirement in section['requirements']:
        try:
            logger.info(f"Generating test for requirement: {requirement['id']}")
            
            # Prepare the prompt for this requirement with full context
            req_prompt = get_inferno_test_generation_prompt(
                test_specification=requirement['full_content'],
                requirement_id=requirement['id'],
                module_name=module_name,
                inferno_guidance=inferno_guidance
            )
            
            # Generate the test
            test_code = llm_utils.make_llm_request(
                client, 
                api_type, 
                req_prompt, 
                INFERNO_TEST_SYSTEM_PROMPT, 
                rate_limit_func
            )
            
            # Clean up any markdown formatting
            if test_code.startswith('```ruby'):
                test_code = re.sub(r'^```ruby\n', '', test_code)
                test_code = re.sub(r'\n```$', '', test_code)
            elif test_code.startswith('```'):
                test_code = re.sub(r'^```\n', '', test_code)
                test_code = re.sub(r'\n```$', '', test_code)
            
            tests[requirement['id']] = test_code
            logger.info(f"Successfully generated test for requirement: {requirement['id']}")
            
        except Exception as e:
            logger.error(f"Error generating test for requirement {requirement['id']}: {str(e)}")
    
    return tests

In [269]:
def create_file_structure(grouped_tests: Dict[str, Dict[str, List[Dict[str, Any]]]], module_name: str, output_dir: str):
    """
    Create the file structure for the Inferno tests
    
    Args:
        grouped_tests: Nested dictionary of grouped test specs
        module_name: Name of the module for the tests
        output_dir: Directory for output files
        
    Returns:
        Dictionary mapping test specs to file paths
    """
    # Create base directories
    module_dir = os.path.join(output_dir, module_name.lower())
    os.makedirs(module_dir, exist_ok=True)
    
    # Create test group directories
    test_file_map = {}
    
    for actor, resource_groups in grouped_tests.items():
        actor_dir = os.path.join(module_dir, actor.lower())
        os.makedirs(actor_dir, exist_ok=True)
        
        for resource, test_specs in resource_groups.items():
            resource_dir = os.path.join(actor_dir, resource.lower())
            os.makedirs(resource_dir, exist_ok=True)
            
            for spec in test_specs:
                # Create a file name based on the requirement ID
                req_id = spec['id'].lower().replace('-', '_')
                file_name = f"{req_id}_test.rb"
                file_path = os.path.join(resource_dir, file_name)
                
                test_file_map[spec['id']] = file_path
    
    return test_file_map

In [270]:
def generate_module_file(module_name: str, output_dir: str, grouped_tests: Dict[str, Dict[str, List[Dict[str, Any]]]]):
    """
    Generate the main module file that includes all test groups
    
    Args:
        module_name: Name of the module for the tests
        output_dir: Directory for output files
        grouped_tests: Nested dictionary of grouped test specs
        
    Returns:
        Path to the generated module file
    """
    module_dir = os.path.join(output_dir, module_name.lower())
    module_file_path = os.path.join(module_dir, f"{module_name.lower()}.rb")
    
    # Generate module file content
    module_content = f"# {module_name} Inferno Test Suite\n"
    module_content += "# Generated on: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n\n"
    module_content += "require 'inferno/dsl/test_suite'\n\n"
    
    # Add requires for all test groups
    for actor, resource_groups in grouped_tests.items():
        # Create safe versions for file paths
        actor_safe = re.sub(r'[^a-zA-Z0-9_]', '_', actor.lower())
        
        for resource, test_specs in resource_groups.items():
            # Create safe versions for file paths
            resource_safe = re.sub(r'[^a-zA-Z0-9_]', '_', resource.lower())
            
            module_content += f"# {actor} {resource} tests\n"
            for spec in test_specs:
                req_id = spec['id'].lower().replace('-', '_')
                module_content += f"require_relative '{actor_safe}/{resource_safe}/{req_id}_test'\n"
            module_content += "\n"
    
    # Add module definition
    module_content += f"module {module_name}\n"
    
    # Add test suite class
    class_name = f"{module_name}TestSuite"
    module_content += f"  class {class_name} < Inferno::TestSuite\n"
    module_content += f"    id :{module_name.lower()}_suite\n"
    module_content += f"    title '{module_name} Test Suite'\n"
    module_content += f"    description 'Test suite for validating {module_name} Implementation Guide conformance'\n\n"
    
    # Add actor groups
    for actor, resource_groups in grouped_tests.items():
        # Create safe version for Ruby identifiers
        actor_safe = re.sub(r'[^a-zA-Z0-9_]', '_', actor.lower())
        
        module_content += f"    # {actor} tests\n"
        module_content += f"    group do\n"
        module_content += f"      id :{actor_safe}_group\n"
        module_content += f"      title '{actor} Tests'\n\n"
        
        # Add resource groups
        for resource, test_specs in resource_groups.items():
            # Skip empty resource groups
            if not test_specs:
                continue
                
            # Create safe version for Ruby identifiers
            resource_safe = re.sub(r'[^a-zA-Z0-9_]', '_', resource.lower())
            
            module_content += f"      # {resource} tests\n"
            module_content += f"      group do\n"
            module_content += f"        id :{actor_safe}_{resource_safe}_group\n"
            module_content += f"        title '{resource} Tests'\n\n"
            
            # Add references to individual tests
            for spec in test_specs:
                req_id = spec['id'].lower().replace('-', '_')
                module_content += f"        test from: :{req_id}_test\n"
            
            module_content += "      end\n\n"
        
        module_content += "    end\n\n"
    
    module_content += "  end\n"
    module_content += "end\n"
    
    # Write to file
    with open(module_file_path, 'w') as f:
        f.write(module_content)
    
    return module_file_path

In [271]:
def generate_inferno_test_kit(
    api_type: str,
    test_plan_file: str,
    guidance_file: str = None,
    module_name: str = "PlanNet",
    output_dir: str = OUTPUT_DIR,
    expected_actors: List[str] = None
) -> Dict[str, Any]:
    """
    Process a test plan and generate an Inferno test kit
    
    Args:
        api_type: API type (claude, gemini, gpt)
        test_plan_file: Path to test plan markdown file
        guidance_file: Path to Inferno guidance file (optional)
        module_name: Name of the module for the tests
        output_dir: Directory for output files
        expected_actors: List of expected actors in the test plan
        
    Returns:
        Dictionary containing statistics and paths
    """
    logger.info(f"Starting Inferno test generation with {api_type} for {module_name}")
    
    # Initialize API clients and rate limiters
    clients = llm_utils.setup_clients()
    client = clients[api_type]
    config = llm_utils.API_CONFIGS[api_type]
    rate_limiter = llm_utils.create_rate_limiter()
    
    def check_limits():
        llm_utils.check_rate_limits(rate_limiter, api_type)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Module name normalization for Ruby
    module_name = ''.join(word.capitalize() for word in module_name.split())
    
    # Set default actors if none provided
    if not expected_actors:
        expected_actors = ["Health Plan API Actor", "Application Actor"]
    
    try:
        # Parse test plan into sections
        sections = parse_test_plan(test_plan_file)
        logger.info(f"Parsed test plan into {len(sections)} sections")
        
        # Count total requirements
        total_requirements = sum(len(section['requirements']) for section in sections.values())
        logger.info(f"Found {total_requirements} total requirements")
        
        # Get Inferno guidance
        if guidance_file and os.path.exists(guidance_file):
            with open(guidance_file, 'r') as f:
                inferno_guidance = f.read()
            logger.info(f"Loaded Inferno guidance from {guidance_file}")
        else:
            inferno_guidance = get_inferno_guidance()
            logger.info("Using default Inferno guidance")
        
        # Generate tests by section
        all_tests = {}
        for section_name, section in sections.items():
            logger.info(f"Processing section: {section_name} with {len(section['requirements'])} requirements")
            
            # Skip empty sections
            if not section['requirements']:
                continue
                
            # Generate tests for this section
            section_tests = generate_tests_for_section(
                client, 
                api_type, 
                section, 
                inferno_guidance, 
                module_name, 
                check_limits,
                config['max_tokens']
            )
            
            # Add to our collection
            all_tests.update(section_tests)
            
            # Add delay between sections
            time.sleep(config["delay_between_batches"])
            
        logger.info(f"Generated tests for {len(all_tests)} requirements")
        
        # Group requirements by actor and section (original groups from test plan)
        grouped_reqs = determine_test_groups(sections, expected_actors)
        
        # Ensure all expected actors are represented, even if empty
        for actor in expected_actors:
            if actor not in grouped_reqs:
                grouped_reqs[actor] = {}
        
        # Create file structure and write tests
        os.makedirs(os.path.join(output_dir, module_name.lower()), exist_ok=True)
        
        file_paths = {}
        for actor, section_groups in grouped_reqs.items():
            actor_safe = re.sub(r'[^a-zA-Z0-9_]', '_', actor.lower())
            actor_dir = os.path.join(output_dir, module_name.lower(), actor_safe)
            os.makedirs(actor_dir, exist_ok=True)
            
            for section_name, reqs in section_groups.items():
                section_safe = re.sub(r'[^a-zA-Z0-9_]', '_', section_name.lower())
                section_dir = os.path.join(actor_dir, section_safe)
                os.makedirs(section_dir, exist_ok=True)
                
                for req in reqs:
                    # Skip requirements we couldn't generate tests for
                    if req['id'] not in all_tests:
                        continue
                        
                    file_name = f"{req['id'].lower().replace('-', '_')}_test.rb"
                    file_path = os.path.join(section_dir, file_name)
                    
                    with open(file_path, 'w') as f:
                        f.write(all_tests[req['id']])
                    
                    file_paths[req['id']] = file_path
                    logger.info(f"Wrote test for {req['id']} to {file_path}")
        
        # Generate module file
        module_file = generate_module_file(module_name, output_dir, grouped_reqs)
        
        return {
            "total_sections": len(sections),
            "total_requirements": total_requirements,
            "generated_tests": len(all_tests),
            "module_dir": os.path.join(output_dir, module_name.lower()),
            "module_file": module_file
        }
        
    except Exception as e:
        logger.error(f"Error generating Inferno tests: {str(e)}")
        raise

In [272]:
def generate_module_file(module_name: str, output_dir: str, grouped_reqs) -> str:
    """
    Generate the main module file that includes all test groups
    
    Args:
        module_name: Name of the module for the tests
        output_dir: Directory for output files
        grouped_reqs: Dictionary of requirements grouped by actor and section
        
    Returns:
        Path to the generated module file
    """
    module_dir = os.path.join(output_dir, module_name.lower())
    module_file_path = os.path.join(module_dir, f"{module_name.lower()}.rb")
    
    # Generate module file content
    module_content = f"# {module_name} Inferno Test Suite\n"
    module_content += "# Generated on: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n\n"
    module_content += "require 'inferno/dsl/test_suite'\n\n"
    
    # Add requires for all test groups
    for actor, section_groups in grouped_reqs.items():
        actor_safe = actor.replace(" ", "_").lower()
        for section_name, reqs in section_groups.items():
            section_safe = section_name.replace(" ", "_").lower()
            module_content += f"# {actor} - {section_name} tests\n"
            for req in reqs:
                req_id_safe = req['id'].lower().replace('-', '_')
                module_content += f"require_relative '{actor_safe}/{section_safe}/req_{req_id_safe}_test'\n"
            module_content += "\n"
    
    # Add module definition
    module_content += f"module {module_name}\n"
    
    # Add test suite class
    class_name = f"{module_name}TestSuite"
    module_content += f"  class {class_name} < Inferno::TestSuite\n"
    module_content += f"    id :{module_name.lower()}_suite\n"
    module_content += f"    title '{module_name} Test Suite'\n"
    module_content += f"    description 'Test suite for validating {module_name} Implementation Guide conformance'\n\n"
    
    # Add actor groups
    for actor, section_groups in grouped_reqs.items():
        actor_safe = actor.replace(" ", "_").lower()
        module_content += f"    # {actor} tests\n"
        module_content += f"    group do\n"
        module_content += f"      id :{actor_safe}_group\n"
        module_content += f"      title '{actor} Tests'\n\n"
        
        # Add section groups
        for section_name, reqs in section_groups.items():
            section_safe = section_name.replace(" ", "_").lower()
            module_content += f"      # {section_name} tests\n"
            module_content += f"      group do\n"
            module_content += f"        id :{actor_safe}_{section_safe}_group\n"
            module_content += f"        title '{section_name} Tests'\n\n"
            
            # Add references to individual tests
            for req in reqs:
                req_id_safe = req['id'].lower().replace('-', '_')
                module_content += f"        test from: :req_{req_id_safe}_test\n"
            
            module_content += "      end\n\n"
        
        module_content += "    end\n\n"
    
    module_content += "  end\n"
    module_content += "end\n"
    
    # Write to file
    with open(module_file_path, 'w') as f:
        f.write(module_content)
    
    return module_file_path

### Main Execution Function

In [273]:
def run_inferno_test_generator():
    """
    Run the Inferno test generator with user input
    
    Returns:
        Dictionary with generation results, or None if an error occurred
    """
    # Load environment variables
    load_dotenv()
    
    # Get input from user or set default values
    print("\nFHIR Inferno Test Generator")
    print("=" * 50)
    
    # Let user select the API
    print("\nSelect the API to use:")
    print("1. Claude")
    print("2. Gemini")
    print("3. GPT-4")
    api_choice = input("Enter your choice (1-3, default 1): ") or "1"
    
    api_mapping = {
        "1": "claude",
        "2": "gemini",
        "3": "gpt"
    }
    
    api_type = api_mapping.get(api_choice, "claude")
    
    # Get test plan file path
    test_plan_file = input("\nEnter path to test plan markdown file: ")
    
    # Check if test plan file exists
    if not os.path.exists(test_plan_file):
        logger.error(f"Test plan file not found: {test_plan_file}")
        print(f"Error: Test plan file not found at {test_plan_file}")
        return None
    
    # the path to the guidance file
    guidance_file = os.path.join(CURRENT_DIR, "inferno-guidance.md")
    
    # Check if guidance file exists
    if not os.path.exists(guidance_file):
        logger.warning(f"Inferno guidance file not found: {guidance_file}")
        print(f"Warning: Inferno guidance file not found at {guidance_file}. Using built-in guidance.")
        guidance_file = None
    else:
        print(f"Using guidance file: {guidance_file}")
    
    # Get module name
    module_name = input("\nEnter module name (default 'PlanNet'): ") or "PlanNet"
    
    # Get actor information
    print("\nEnter the actors from the test plan (comma-separated, e.g., 'Health Plan API Actor, Application Actor'):")
    actors_input = input("Actors: ")
    
    if actors_input:
        expected_actors = [actor.strip() for actor in actors_input.split(',')]
    else:
        # Default actors if none provided
        expected_actors = ["Health Plan API Actor", "Application Actor"]
    
    # Get output directory
    output_dir = input(f"\nEnter output directory (default '{OUTPUT_DIR}'): ") or OUTPUT_DIR
    
    print(f"\nGenerating Inferno tests with {api_type.capitalize()}...")
    print(f"Using actors: {', '.join(expected_actors)}")
    if guidance_file:
        print(f"Using Inferno guidance from {guidance_file}")
    print(f"This may take several minutes depending on the number of requirements.")
    
    try:
        # Process test plan and generate Inferno tests
        result = generate_inferno_test_kit(
            api_type=api_type,
            test_plan_file=test_plan_file,
            guidance_file=guidance_file,
            module_name=module_name,
            output_dir=output_dir,
            expected_actors=expected_actors
        )
        
        # Output results
        print("\n" + "="*80)
        print(f"Inferno test generation complete!")
        print(f"Total sections: {result['total_sections']}")
        print(f"Total requirements: {result['total_requirements']}")
        print(f"Successfully generated tests: {result['generated_tests']}")
        print(f"Module directory: {result['module_dir']}")
        print(f"Main module file: {result['module_file']}")
        #print(f"Generation report: {result['report_file']}")
        print("="*80)
        
        return result
        
    except Exception as e:
        logger.error(f"Error: {str(e)}")
        print(f"\nError occurred during processing: {str(e)}")
        print("Check the log for more details.")
        return None

In [274]:
# Run the test generator
run_inferno_test_generator()


FHIR Inferno Test Generator

Select the API to use:
1. Claude
2. Gemini
3. GPT-4
Using guidance file: /Users/ceadams/Documents/onclaive/onclaive/test_kit_dev/inferno-guidance.md

Enter the actors from the test plan (comma-separated, e.g., 'Health Plan API Actor, Application Actor'):


2025-04-30 12:34:41,605 - __main__ - INFO - Starting Inferno test generation with gemini for PlanNet
2025-04-30 12:34:41,640 - __main__ - INFO - Parsed test plan into 11 sections
2025-04-30 12:34:41,641 - __main__ - INFO - Found 11 total requirements
2025-04-30 12:34:41,642 - __main__ - INFO - Loaded Inferno guidance from /Users/ceadams/Documents/onclaive/onclaive/test_kit_dev/inferno-guidance.md
2025-04-30 12:34:41,642 - __main__ - INFO - Processing section: Application-Level Requirements with 1 requirements
2025-04-30 12:34:41,642 - __main__ - INFO - Generating tests for section: Application-Level Requirements
2025-04-30 12:34:41,643 - __main__ - INFO - Generating test for requirement: REQ-08



Generating Inferno tests with Gemini...
Using actors: Health Plan API, Application Actor
Using Inferno guidance from /Users/ceadams/Documents/onclaive/onclaive/test_kit_dev/inferno-guidance.md
This may take several minutes depending on the number of requirements.
Found 11 potential requirements
Processing requirement: REQ-08
Added requirement REQ-08 to section Application-Level Requirements
Processing requirement: REQ-01
Added requirement REQ-01 to section Authentication
Processing requirement: REQ-09
Added requirement REQ-09 to section Base Requirements
Processing requirement: REQ-07
Added requirement REQ-07 to section CORE Conformance
Processing requirement: REQ-06
Added requirement REQ-06 to section Cross-Resource
Processing requirement: REQ-04
Added requirement REQ-04 to section General Requirements
Processing requirement: REQ-05
Added requirement REQ-05 to section Global
Processing requirement: REQ-11
Added requirement REQ-11 to section OrganizationAffiliation
Processing requirem

2025-04-30 12:35:38,419 - __main__ - INFO - Successfully generated test for requirement: REQ-08
2025-04-30 12:35:43,429 - __main__ - INFO - Processing section: Authentication with 1 requirements
2025-04-30 12:35:43,430 - __main__ - INFO - Generating tests for section: Authentication
2025-04-30 12:35:43,432 - __main__ - INFO - Generating test for requirement: REQ-01
2025-04-30 12:36:06,273 - __main__ - INFO - Successfully generated test for requirement: REQ-01
2025-04-30 12:36:11,278 - __main__ - INFO - Processing section: Base Requirements with 1 requirements
2025-04-30 12:36:11,278 - __main__ - INFO - Generating tests for section: Base Requirements
2025-04-30 12:36:11,279 - __main__ - INFO - Generating test for requirement: REQ-09
2025-04-30 12:37:18,694 - __main__ - INFO - Successfully generated test for requirement: REQ-09
2025-04-30 12:37:23,694 - __main__ - INFO - Processing section: CORE Conformance with 1 requirements
2025-04-30 12:37:23,695 - __main__ - INFO - Generating tests 


Inferno test generation complete!
Total sections: 11
Total requirements: 11
Successfully generated tests: 11
Module directory: /Users/ceadams/Documents/onclaive/onclaive/test_kit_dev/test_output/plannet
Main module file: /Users/ceadams/Documents/onclaive/onclaive/test_kit_dev/test_output/plannet/plannet.rb


{'total_sections': 11,
 'total_requirements': 11,
 'generated_tests': 11,
 'module_dir': '/Users/ceadams/Documents/onclaive/onclaive/test_kit_dev/test_output/plannet',
 'module_file': '/Users/ceadams/Documents/onclaive/onclaive/test_kit_dev/test_output/plannet/plannet.rb'}